In [126]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [127]:
from diem.io import read_diem_bed
from diem.io import write_polarized_bed
from diem import diemtype as dt

In [128]:
bedIn = '/home/derek/Dropbox/methods/diempy_example_data/CM001010.3_variants.vcf.diem_input.bed'
metaIn = '/home/derek/Dropbox/methods/diempy_example_data/CM001010.3_variants.vcf.diem_meta.bed'
bedOut = '/home/derek/Dropbox/methods/diempy_example_data/CM001010.3_variants.vcf.diem_output.bed'
d = read_diem_bed(bedIn,metaIn)
d.__dict__

{'DMBC': [array([[3, 3, 3, ..., 3, 3, 0],
         [1, 1, 1, ..., 1, 1, 0],
         [3, 3, 3, ..., 2, 3, 0],
         ...,
         [3, 3, 3, ..., 3, 3, 3],
         [3, 3, 3, ..., 2, 3, 0],
         [1, 1, 1, ..., 1, 1, 1]], shape=(10, 961395), dtype=int8)],
 'indNames': array(['swn', 'pgn', 'msm2013', 'njl', 'jf1', 'hmi', 'chd', 'blg2', 'kjr',
        'bfm'], dtype=object),
 'chrPloidies': [array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])],
 'chrNames': array(['CM001010.3'], dtype=object),
 'posByChr': [array([ 3050186,  3050775,  3051166, ..., 95194214, 95194318, 95194506],
        shape=(961395,))],
 'chrLengths': array([95294699]),
 'MapBC': [array([0.03200793, 0.03201411, 0.03201821, ..., 0.99894553, 0.99894663,
         0.9989486 ], shape=(961395,))],
 'HIs': None,
 'initialPolByChr': None,
 'PolByChr': None,
 'DIByChr': None,
 'SupportByChr': None,
 'threshold': None,
 'smoothScale': None,
 'contigMatrix': None,
 'siteExclusionsByChr': None,
 'indExclusions': None}

When polarizing a diemtypeobject, it RETURNS a POLARIZED COPY

In [65]:
d.add_individual_exclusions('/home/derek/Dropbox/methods/diempy_example_data/example_individual_exclusions.txt')

In [66]:
d.add_site_exclusions('/home/derek/Dropbox/methods/diempy_example_data/example_site_exclusions.bed')

In [67]:
d.indExclusions

array(['hmi', 'chd'], dtype='<U3')

In [69]:
dPol = d.polarize(ncores = 3,boolTestData=True)

convert state matrix to Marray
initializing test polarity
running EM algorithm in parallel using diem_parallel with  3  cores
starting
using  3  CPUs
iteration  0   num 0 polarity =  480698   num 1 polarity =  480697
time to flip:  0.7352495193481445
iteration  1   num 0 polarity =  304481   num 1 polarity =  656914
time to flip:  0.6320908069610596
iteration  2   num 0 polarity =  189243   num 1 polarity =  772152
time to flip:  0.6594803333282471
iteration  3   num 0 polarity =  224186   num 1 polarity =  737209
time to flip:  0.6737701892852783
iteration  4   num 0 polarity =  225401   num 1 polarity =  735994
time to flip:  0.6149616241455078
iteration  5   num 0 polarity =  225468   num 1 polarity =  735927
time to flip:  0.6682751178741455
iteration  6   num 0 polarity =  225473   num 1 polarity =  735922
time to flip:  0.6419596672058105
polarity found
polarity found
updating polarizations, DIs, Supports, initialPolarity,and state matrices


This is a (currently slow way) to output that polarization information to an output.bed file.  It reads in the original, appends the additional columns, then writes it all out again.

This makes it easy because even if we have somehow resorted the individuals by hybrid index, that has no effect on the output file. It also means we keep all the state information that is removed (that is, converted to '_' or 0 state internal to diemtype object)

In [73]:
write_polarized_bed(bedIn,bedOut,dPol)

We now want to check that we can correctly read in the polarized output version of the diem_bed file

In [116]:
dPolInTest = read_diem_bed(bedOut,metaIn)

Most attributes are fine, as you can see in the comparison below.
1. The HIs are not stored, but can calculated again on import easy-peasy
2. DIByChr and SupportByChr are only off by a rounding of the last digit. should be easily fixed

In [121]:
from diem.tests import compare_DiemTypes
compare_DiemTypes(dPol,dPolInTest)

Now checking list attributes
checking DMBC
checking PolByChr
checking DIByChr
Attribute  DIByChr  differs for chromosome  0
checking SupportByChr
Attribute  SupportByChr  differs for chromosome  0
checking posByChr
checking MapBC
checking chrPloidies
checking initialPolByChr
Now checking non-list attributes
Attribute  HIs  differs (one is None, the other is not)
Now checking float attributes
now checking individual exclusions
now checking site exclusions
previous attributes failed, so skipping contigMatrix check
The following attributes differed:  ['DIByChr', 'SupportByChr', 'HIs']


In [124]:
print(
    dPol.DIByChr[0][0],
    dPolInTest.DIByChr[0][0]
)

-0.37666822243435244 -0.3766682224343524


Thresholding RETURNS A COPY of the diemtype object with sites removed that are below threshold

In [ ]:
dThresh = dPol.apply_threshold(-20)


proportion of each chromosome retained after thresholding:
[np.float64(0.8675965654075588)]
thresholding done
hybrid indices have likely changed, but have not been updated
to update HIs without re-sorting, update the attribute a.HIs = a.computeHIs()
You may also call the sort() method on the resulting data if you wish to both re-compute and resort


SORTING IS DONE IN-PLACE
meaning that the diemtypeobject itself is sorted. Nothing is returned.
To obtain a sorted *copy* you must first copy it then sort as follows:

In [11]:
dSorted = dThresh.copy()
dSorted.sort()

new HIs computer and individuals resorted by HI


Smoothing RETURNS A COPY

In [13]:
dSmoothed = dSorted.smooth(1e-4,parallel=True)

smoothing the data with scale  0.0001
using parallel smoothing
smoothing done, but not resorting by HI. You may call the sort() method on the resulting data if you wish to resort


Constructing contigs adds this information to the diemtype objects
Meaning that it DOES NOT RETURN A COPY


In [ ]:
dSmoothed.create_contig_matrix()

contig matrix created and stored in self.contigMatrix


We provide a path to a *folder* where we want the intervals information to be written to individual-specific bed files. 

In [17]:
outIntervalsFolder = '/home/derek/Dropbox/methods/diempy_example_data/CM001010_intervals/'
dSmoothed.intervals_to_bed(outIntervalsFolder)

BED files created in directory:  /home/derek/Dropbox/methods/diempy_example_data/CM001010_intervals/
